In [ ]:
%pylab inline
%config InlineBackend.figure_formats = ['retina']

import pandas as pd
import seaborn as sns
sns.set()

In [ ]:
import sqlite3 as sq3
import pandas.io.sql as pds
import pandas as pd
# standard
import numpy as np
import pandas as pd
import time

# plot
import matplotlib.pyplot as plt
import seaborn as sns

# statistics tools
from statsmodels.graphics.mosaicplot import mosaic

import os
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import KFold, cross_val_predict
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import r2_score
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error

In [ ]:

cd Users/syedwaqar/Huma # to change directory 

In [ ]:
pwd # to check directory

In [ ]:
path = 'healthcare-dataset-stroke-data.csv'
con = sq3.Connection(path)

In [ ]:


os.path.isfile('healthcare-dataset-stroke-data.csv')

In [ ]:
con

In [ ]:
pd.DataFrame([[1,0,5],[0,1,4]])



In [ ]:
data = pd.read_csv(path)

In [ ]:
data.head()

In [ ]:
os.sep # to check the separator used for speciifying the file location


In [ ]:
#feature engineering 
data.info()

In [ ]:
# Get a Pd.Series consisting of all the string categoricals
one_hot_encode_cols = data.dtypes[data.dtypes == np.object]  # filtering by string categoricals
one_hot_encode_cols = one_hot_encode_cols.index.tolist()  # list of categorical fields

data[one_hot_encode_cols].head().T

In [ ]:
data = pd.get_dummies(data, columns=one_hot_encode_cols, drop_first=True)
data.describe().T

In [ ]:
data.dropna(inplace=True)

In [ ]:
data.info()


In [ ]:

X = data.drop('stroke', axis=1)
y = data.stroke

In [ ]:
kf = KFold(shuffle=True, random_state=72018, n_splits=3)

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
for train_index, test_index in kf.split(X):
    print("Train index:", train_index[:10], len(train_index))
    print("Test index:",test_index[:10], len(test_index))
    print('')

In [ ]:
X.iloc[train_index, :]

In [ ]:
X.iloc[test_index, :]

In [ ]:
y.iloc[train_index]

In [ ]:
#from sklearn.metrics import r2_score, mean_squared_error


scores = []
lr = LinearRegression()

for train_index, test_index in kf.split(X):
    X_train, X_test, y_train, y_test = (X.iloc[train_index, :], 
                                        X.iloc[test_index, :], 
                                        y.iloc[train_index], # imp to use '.iloc' for speciifying y train index 
                                        y.iloc[test_index])
    
    lr.fit(X_train, y_train)
        
    y_pred = lr.predict(X_test)

    score = r2_score(y_test.values, y_pred)
    
    scores.append(score)
    
scores # the closer to 1 is the r2 score, the better are we in explaining the variance

In [ ]:
data.isnull().sum()# only bmi had null vaules and we deleted them 

In [ ]:
mean_squared_error(y_test.values, y_pred)

In [ ]:
mask = data.dtypes == np.float
float_cols = data.columns[mask]

skew_limit = 0.75 # define a limit above which we will log transform
skew_vals = X[float_cols].skew()

In [ ]:
skew_cols = (skew_vals
             .sort_values(ascending=False)
             .to_frame()
             .rename(columns={0:'Skew'})
             .query('abs(Skew) > {}'.format(skew_limit)))

skew_cols

In [ ]:
# Choose a field
field = ["avg_glucose_level", "bmi"]


# Create two "subplots" and a "figure" using matplotlib
fig, (ax_before, ax_after) = plt.subplots(2, 2, figsize=(10, 5))

# Create a histogram on the "ax_before" subplot
X[field].hist(ax=ax_before)

# Apply a log transformation (numpy syntax) to this column
X[field].apply(np.log1p).hist(ax=ax_after)
# Formatting of titles etc. for each subplot
ax_before.set(title='before np.log1p', ylabel='frequency', xlabel='value')
ax_after.set(title='after np.log1p', ylabel='frequency', xlabel='value')
fig.suptitle('Field "{}"'.format(field));

In [ ]:
smaller_df= X.loc[:,field]

In [ ]:
# Now we can look at summary statistics of the subset data
smaller_df.describe().T

In [ ]:
smaller_df.info()

In [ ]:
sns.pairplot(smaller_df, plot_kws=dict(alpha=.1, edgecolor='none')) 

In [ ]:
#from sklearn.metrics import r2_score, mean_squared_error


scores = []
lr = LinearRegression()

for train_index, test_index in kf.split(X):
    X_train, X_test, y_train, y_test = (X.iloc[train_index, :], 
                                        X.iloc[test_index, :], 
                                        y.iloc[train_index], # imp to use '.iloc' for speciifying y train index 
                                        y.iloc[test_index])
    
    lr.fit(X_train, y_train)
        
    y_pred = lr.predict(X_test)

    score = r2_score(y_test.values, y_pred)
    
    scores.append(score)
    
scores # the closer to 1 is the r2 score, the better are we in explaining the variance

In [ ]:
mean_squared_error(y_test.values, y_pred)

In [ ]:
scores = []

lr = LinearRegression()
s = StandardScaler()

for train_index, test_index in kf.split(X):
    X_train, X_test, y_train, y_test = (X.iloc[train_index, :], 
                                        X.iloc[test_index, :], 
                                        y.iloc[train_index], 
                                        y.iloc[test_index])
    
    X_train_s = s.fit_transform(X_train)
    
    lr.fit(X_train_s, y_train)
    
    X_test_s = s.transform(X_test)
    
    y_pred = lr.predict(X_test_s)

    score = r2_score(y_test.values, y_pred)
    
    scores.append(score)

In [ ]:
scores # these have improved very minutely and have increased little bit from 
# [0.08851187144854655, 0.060816944650728955, 0.06733740589360682] to below values

In [ ]:
mean_squared_error(y_test.values, y_pred) # error has decresead very minutely from '0.039239837981126346' to below 

In [ ]:
#hyperparameter tuning with scaling, polynomial 

pf = PolynomialFeatures()
mse = []
scores = []
alphas = np.geomspace(1e-9, 1e0, num=10)
for alpha in alphas:
    las = Lasso(alpha=alpha, max_iter=100000)
    
    estimator = Pipeline([
        ("scaler", s),
        ("make_higher_degree", pf),
        ("lasso_regression", las)])

    predictions = cross_val_predict(estimator, X, y, cv = kf)
    
    score = r2_score(y, predictions)
    m_s_e = mean_squared_error(y, predictions)
    scores.append(score)
    mse.append(m_s_e)

In [ ]:
list(zip(alphas,scores)) # here we can see that 0.06 is the best alpha as it gives us the most score 

In [ ]:
plt.semilogx(alphas, scores);

In [ ]:
list(zip(alphas,mse))

In [ ]:
plt.semilogx(alphas, mse);

In [ ]:
from sklearn.model_selection import GridSearchCV

# Same estimator as before
estimator = Pipeline([("scaler", StandardScaler()),
        ("polynomial_features", PolynomialFeatures()),
        ("lasso_regression", Lasso(alpha=0.01))])

params = {
    'polynomial_features__degree': [1, 2, 3]
}

grid = GridSearchCV(estimator, params, cv=kf)

In [ ]:
grid.fit(X, y)

In [ ]:
grid.best_score_, grid.best_params_

In [ ]:
y_predict = grid.predict(X)

In [ ]:
r2_score(y, y_predict)

In [ ]:
mean_squared_error(y, y_predict)

In [ ]:
grid.best_estimator_.named_steps['lasso_regression'].coef_

In [ ]:
grid.cv_results_

In [ ]:
grid.best_estimator_.named_steps["polynomial_features"].get_feature_names(input_features=X.columns)

In [ ]:
data_importances = pd.DataFrame(zip(grid.best_estimator_.named_steps["polynomial_features"].get_feature_names(input_features=X.columns),
                 grid.best_estimator_.named_steps["lasso_regression"].coef_,
))

In [ ]:
col_names_dict = dict(zip(list(range(len(X.columns.values))), X.columns.values))

In [ ]:
col_names_dict

In [ ]:
data_importances.sort_values(by=1)

In [ ]:
from sklearn.model_selection import GridSearchCV

# Same estimator as before
estimator = Pipeline([("scaler", StandardScaler()),
        ("polynomial_features", PolynomialFeatures()),
        ("ridge_regression", Ridge())])

params = {
    'polynomial_features__degree': [1, 2, 3],
    'ridge_regression__alpha': np.geomspace(1e-9, 1e0, num=10)
}

grid = GridSearchCV(estimator, params, cv=kf)

In [ ]:
grid.fit(X, y)

In [ ]:
grid.best_score_, grid.best_params_

In [ ]:
y_predict = grid.predict(X)

In [ ]:
r2_score(y, y_predict)

In [ ]:
mean_squared_error(y, y_predict)

In [ ]:
grid.best_estimator_.named_steps["polynomial_features"].get_feature_names(input_features=X.columns)

In [ ]:
data_importances = pd.DataFrame(zip(grid.best_estimator_.named_steps["polynomial_features"].get_feature_names(input_features=X.columns),
                 grid.best_estimator_.named_steps["ridge_regression"].coef_,
))

In [ ]:
col_names_dict = dict(zip(list(range(len(X.columns.values))), X.columns.values))

In [ ]:
col_names_dict 

In [ ]:
data_importances.sort_values(by=1)

In [ ]:
fig = plt.figure(figsize=(30,30))
sns.pairplot(data)
plt.show()

In [ ]:
#After checking the models based on the error values calculated. I found that lasso model was the best suitable model